<a href="https://colab.research.google.com/github/TatanPerez/Senales_Sistemas/blob/main/Dashboard_punto_2_parcial3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit -q #instalación de librerías
!pip install pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.1 MB/s eta 0:00:00


In [ ]:
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install soundfile
!pip install pydub
!pip install --upgrade youtube_dl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 15.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2024.8.30-py3-none-any.whl (167 kB)
Using cached urllib3-2.2.3-py3-none-any.whl (126 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━

In [ ]:
%%writefile Demodulacion2.py
import joblib
import gdown
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import cosine
from scipy.signal import butter, lfilter,freqz,bode,sosfilt, hilbert
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import os
import youtube_dl
from scipy.io import wavfile
import subprocess
from pyngrok import ngrok
import joblib
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import cdist
import gdown
from scipy.signal import butter as bw
from scipy.signal import freqz_zpk
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.patches import Circle
from scipy.signal import zpk2tf, lfilter
@st.cache_data
def download_audio(link, output_name="audio.mp3"):
            command = [
                "yt-dlp",
                "--extract-audio",
                "--audio-format", "mp3",
                "-o", output_name,
                link
            ]
            subprocess.run(command, check=True)
def convert_mp3_to_wav(input_file="audio.mp3", output_file="output.wav"):
    command = [
        "ffmpeg",
        "-y",
        "-i", input_file,
        output_file
    ]
    subprocess.run(command, check=True)
#def download_ytvid_as_mp3(video_url, name):
    #video_info = youtube_dl.YoutubeDL().extract_info(url=video_url, download=False)
    #filename = f"{name}.mp3"
    #options = {
     #   'format': 'bestaudio/best',
      #  'keepvideo': False,
       # 'outtmpl': filename,
    #}

    #ydl = youtube_dl.YoutubeDL(options)
    #ydl.download([video_info['webpage_url']])

    #return filename

st.sidebar.title("Involucrados en el Proyecto")
st.sidebar.write("Juan Jeronimo Castaño Rivera")
st.sidebar.write("Daniel Mauricio Mejia Hoyos")
st.sidebar.write("Wilmer Sebastian Perez Cuastumal")

# Frecuencia portadora y mensaje
Fc = 100
Fm = 10
Am = 1

ti = 0
tf = 5 / Fm
Fs = 5 * Fc

t = np.arange(ti, tf, 1 / Fs)
m = Am * np.cos(2 * np.pi * Fm * t)  # Mensaje

Im = 0.8
Ac = max(abs(m)) / Im  # Amplitud de la portadora
c = Ac * np.sin(2 * np.pi * Fc * t)  # Portadora

# Señal modulada en AM
y = (1 + m / Ac) * c

# Transformadas de Fourier
Xfc = np.fft.fft(c)
Xfm = np.fft.fft(m)
Xfy = np.fft.fft(y)
vfre = np.fft.fftfreq(len(c), 1 / Fs)


st.title("Modulación AM, Demodulacion y Detección de género ")
st.subheader("Modulación AM")
st.markdown("""
El proceso de modulación, usado principalmente en telecomunicaciones, es un proceso usado en la transmisión de señales por un medio (ya sea físico o no).

Se tiene en este caso la señal que se quiere transmitir (señal mensaje). Estas señales son principalmente de baja frecuencia, por lo que para transmitirlas de un lado a otro, se requieren de equipos muy especializados y en tanto, costosos, por lo que se aplica esta técnica.

La señal mensaje se mezcla con una señal portadora (puede ser una señal senoidal), de muy alta frecuencia, que la va a transportar por el medio.

Al mezclarlas, la señal mensaje modula a partir de su amplitud a la señal de alta frecuencia, haciendo que el mensaje original quede inscrito allí y se pueda transmitir fácilmente debido a que sigue conservando la alta frecuencia de la señal portadora
""")
st.markdown("""
- Se requiere entonces de una portadora (carrier) senoidal/cosenoidal:
""")
st.latex("c(t)=A_c\sin(2\pi F_c t)")
st.markdown("""
- Para ilustrar el proceso, se trabaja con un mensaje tipo sen/cos:
""")
st.latex("m(t) = A_m \cos(2\pi F_m t)")
st.markdown("""
- La modulación AM puede calibrarse en términos del índice de modulación:
""")
st.latex("I_m = \\frac{\\text{Peak}\{|m(t)|\}}{A_c}")
st.markdown("""
donde :
""")
st.latex(r"\text{Peak}\{|m(t)|\} \text{es el valor pico (máximo) de}\,|m(t)|")
st.markdown("""
- La señal a transmitir, señal modulada, en AM se cálcula como:
""")
st.latex("y(t)= (1+\\frac{m(t)}{A_c})c(t)")
st.subheader("Señales en el tiempo")

st.markdown("""
Gráfica portadora y mensaje
""")
plt.figure()
plt.plot(t, c, label='Portadora')
plt.plot(t, m, label='Mensaje')
plt.xlabel('$t[s]$')
plt.ylabel('Amplitud')
plt.legend()
st.pyplot(plt.gcf())

st.subheader("""
Gráfica de la señal modulada
""")
st.markdown("""Se modula la señal a partir de la señal mensaje y señal portadora
""")
plt.figure()
plt.plot(t, y, label='Señal Modulada')
plt.xlabel('$t[s]$')
plt.ylabel('Amplitud')
plt.legend()
st.pyplot(plt.gcf())
st.subheader("A partir de la fft se calcula el espectro de la señal portadora, mensaje y modulada")
st.markdown("""Espectro de portadora""")


plt.figure()
plt.plot(vfre, abs(Xfc), label='Portadora')
plt.xlim([0, 2 * Fc])
plt.xlabel('$f [Hz]$')
plt.ylabel('$|X(f)|$')
plt.legend()
st.pyplot(plt.gcf())
st.markdown("""Espectro de portadora""")

plt.figure()
plt.plot(vfre, abs(Xfm), label='Mensaje')
plt.xlim([0, 2 * Fc])
plt.xlabel('$f [Hz]$')
plt.ylabel('$|X(f)|$')
plt.legend()
st.pyplot(plt.gcf())
st.markdown("""Espectro de portadora""")

plt.figure()
plt.plot(vfre, abs(Xfy), label='Modulado')
plt.xlim([0, 2 * Fc])
plt.xlabel('$f [Hz]$')
plt.ylabel('$|X(f)|$')
plt.legend()
st.pyplot(plt.gcf())

st.title("Demodulación de Señales")


st.write("""
La demodulación es el proceso inverso a la modulación. El objetivo es recuperar la señal de mensaje
m(t) a partir de la señal modulada y(t).
""")


st.header("Pasos Clave de la Demodulación")


st.subheader("1. Multiplicación por la portadora")
st.write("""
Se multiplica la señal modulada por una copia de la portadora para desmodularla. Esto devuelve la señal a su frecuencia original.
""")


st.subheader("2. Filtrado Pasa-bajas")
st.write("""
Después de la multiplicación, la señal contiene frecuencias de la portadora. Se aplica un filtro pasa-bajas para eliminar estas frecuencias altas y conservar solo las bajas (la señal de mensaje).
""")



st.subheader("3. Escalado")
st.write("""
Se aplica un factor de escalado para ajustar la amplitud de la señal recuperada.
""")


st.header("Ecuación de la señal demodulada")
st.latex(r'''
y_{demod}(t) = |LPF(y(t) \cdot c(t))|
''')
st.markdown("""
### Elección del Filtro Butterworth Pasa-Bajas

En este caso, se utilizó un **filtro Butterworth pasa-bajas** debido a varias razones que benefician la demodulación de la señal de audio:

1. **Suavidad en la Respuesta de Frecuencia**: El filtro Butterworth tiene una respuesta en frecuencia muy suave, lo que significa que no introduce picos ni resonancias no deseadas en la señal. Esto es crucial en aplicaciones de audio, ya que asegura que la calidad del sonido demodulado se mantenga lo más natural posible.

2. **Eficiente para Separar la Señal de Mensaje**: Al demodular la señal, queremos eliminar componentes de alta frecuencia que no son parte de la señal de mensaje original. Un filtro Butterworth pasa-bajas es especialmente eficaz para permitir que las frecuencias de la señal de mensaje pasen, mientras atenúa las frecuencias más altas, asegurando que se recuperen los detalles esenciales del audio.

3. **Control de la Frecuencia de Corte**: La flexibilidad en el diseño del filtro permite ajustar la frecuencia de corte de manera que se adapte a la frecuencia de la señal de mensaje.



""")
st.subheader("Filtro Butterworth de orden 6 y frecuencia de corte 10000Hz")
fs=480000
N = 6 #orden el filtro
Wn = 10000
filt = 'lowpass' #tipo del filtro (pasa-baja, pasa-alta, pasabanda, rechazabanda)
out = 'zpk' #la funcion retorna los polos y ceros junto con la ganancia. Para usar numerador y denominador usar 'ba' junto con freqz
zeros, poles, gain = bw(N, Wn, btype=filt, output=out, fs=fs)
#K = len(Xw[:,0]) #longitud del vector transformada
w, h = freqz_zpk(zeros, poles, gain, fs=fs) #Para indicar la longitud usar worN=K
def plot_freq_response(filter_name, w, h, N): #Mostrar la respuesta en frecuencia del filtro
  fig = plt.figure()
  ax1 = fig.add_subplot(1, 1, 1)
  ax1.set_title(filter_name +' Digital filter frequency response, order= ' + str(N))
  ax1.plot(w, 20 * np.log10(abs(h)), 'b')
  ax1.set_ylabel('Amplitude [dB]', color='b')
  ax1.set_xlabel('Frequency [Hz]')
  ax1.grid()
  ax2 = ax1.twinx()
  angles = np.unwrap(np.angle(h))
  ax2.plot(w, angles, 'g')
  ax2.set_ylabel('Angle [radians]', color='g')
  plt.axis('tight')
  st.pyplot(plt.gcf())
def show_zp(z, p, title= 'Z-plane'): # Mostrar la ubicación de los polos y los zeros
  plt.figure()
  ax = plt.gca()

  ax.plot(np.real(z), np.imag(z), 'bo', fillstyle='none', ms = 10)
  ax.plot(np.real(p), np.imag(p), 'rx', fillstyle='none', ms = 10)
  unit_circle = Circle((0,0), radius=1, fill=False,
                        color='black', ls='solid', alpha=0.9)
  ax.add_patch(unit_circle)
  ax.axvline(0, color='0.7')
  ax.axhline(0, color='0.7')

  plt.title(title)
  plt.xlabel(r'Re{$z$}')
  plt.ylabel(r'Im{$z$}')
  plt.axis('equal')
  plt.xlim((-2, 2))
  plt.ylim((-2, 2))
  plt.grid()
  st.pyplot(plt.gcf())
st.subheader("Respuesta en Frecuencia del Filtro Butterworth")
plot_freq_response('Butterworth', w, h, N)
st.subheader("Polos y Ceros del Filtro Butterworth")
show_zp(zeros, poles, 'Butterworth')


st.title("Detección de Género Musical: Metal vs Música Clásica")

st.markdown("""
La detección de géneros musicales es un área de estudio dentro del procesamiento de señales y el aprendizaje automático. En esta aplicación, utilizaremos la **Transformada Rápida de Fourier Real (RFFT)** para analizar la señal de audio, ya que es más eficiente para señales reales como las de audio. Esto nos permitirá determinar si la señal pertenece al género **Metal** o **Música Clásica**.
""")

st.subheader("¿Qué es la Transformada Rápida de Fourier Real (RFFT)?")
st.markdown("""
La **Transformada Rápida de Fourier Real (RFFT)** es una variante de la Transformada Rápida de Fourier (FFT) que se utiliza específicamente cuando la señal de entrada es real, como en el caso de la mayoría de las señales de audio. Mientras que la FFT calcula tanto las componentes de frecuencia positiva como negativa (necesario para señales complejas), la **RFFT** optimiza este cálculo retornando únicamente las frecuencias positivas, lo que reduce el costo computacional sin perder información importante en señales reales.

La detección de género musical se realiza mediante un modelo preentrenado que clasifica dos géneros: Clásica y Metal.
""")

st.header("Modulación de Audio desde YouTube")

# Entrada de link de YouTube
if 'previous_link' not in st.session_state:
    st.session_state.previous_link = ""

# Entrada de link de YouTube
link = st.text_input("Ingrese el link del video de YouTube")

if link:
    # Si el enlace es diferente al anterior, eliminamos los archivos temporales
    if link != st.session_state.previous_link:
        if os.path.exists("audio.mp3"):
            os.remove("audio.mp3")
        if os.path.exists("output.wav"):
            os.remove("output.wav")

    st.session_state.previous_link = link  # Actualizamos el enlace anterior

if link:
    with st.spinner("Descargando audio..."):
        #audio_file = download_ytvid_as_mp3(link, "audio")
        #os.system(f"ffmpeg -y -i {audio_file} output.wav")
        #!yt-dlp --extract-audio -o "audio" --audio-format mp3 {link}
        download_audio(link)
        #os.system(f"ffmpeg -y -i {audio_file} output.wav")
        #!ffmpeg -y -i audio.mp3 output.wav
        convert_mp3_to_wav("audio.mp3", "output.wav")
        #os.remove(audio_file)

    # Cargar el audio
    audio_file = "output.wav"
    x, fs = sf.read(audio_file)

    # Seleccionar el fragmento a modular
    st.subheader("Selección de Fragmento de Audio")
    start_time = st.slider("Tiempo de inicio (segundos)", 0, int(len(x)/fs) - 5, 20)
    end_time = start_time + 5

    xs = x[int(start_time*fs):int(end_time*fs), 0]
    tt = np.arange(start_time, end_time, 1/fs)

    # Gráfica del fragmento de audio
    plt.figure()
    plt.plot(tt, xs, label="Canal izquierdo")
    plt.xlabel('$t[s]$')
    plt.ylabel('$x(t)$')
    plt.legend()
    st.pyplot(plt.gcf())
    st.markdown("""
    Si el pedazo de la cancion es la señal de mensaje y la señal portadora es:
    """)
    st.latex("c(t)=A_c\sin(2\pi F_c t)")
    st.latex(r"F_c=10000Hz")
    # Definir índice de modulación
    Im = st.slider("Seleccione el índice de modulación", min_value=0.1, max_value=1.0, value=0.8)

    # Frecuencia portadora (fijada en 10000 Hz)
    Fc = 10000

    # Señal modulada en AM
    Ac = max(abs(xs)) / Im
    t = np.arange(start_time, end_time, 1/fs)
    c = Ac * np.sin(2 * np.pi * Fc * t)
    y = (1 + xs / Ac) * c

    # Espectro de Frecuencia
    Xfc = np.fft.rfft(c)
    Xfm = np.fft.rfft(xs)
    Xfy = np.fft.rfft(y)
    vfre = np.fft.rfftfreq(len(c), 1 / fs)
    plt.figure()
    plt.plot(t,c,label='portadora')
    plt.plot(t,xs,label='mensaje')
    plt.legend()
    st.pyplot(plt.gcf())
    st.subheader("Señal modulada en AM")
    plt.figure()
    plt.plot(t,y)
    plt.xlabel('$t[s]$')
    plt.ylabel('$y(t)$')
    st.pyplot(plt.gcf())

    st.subheader("Espectro de Frecuencia del Audio Modulado")
    st.markdown("""- A continuación, se presenta el espectro de la señal mensaje (cancion), portadora y modulada.""")
    # Espectro de la portadora
    plt.figure()
    plt.plot(vfre, abs(Xfc), label='Portadora')
    plt.xlabel('$f [Hz]$')
    plt.ylabel('$|X(f)|$')
    plt.legend()
    st.pyplot(plt.gcf())
    # Espectro del mensaje
    plt.figure(figsize=(8, 6))

    # Subplot 1: Espectro modulado
    plt.subplot(2, 1, 1)
    plt.plot(vfre, abs(Xfy), label='Modulado', color="r")
    plt.xlabel('$f [Hz]$')
    plt.ylabel('$|X(f)|$')
    plt.legend()
    plt.ylim([min(abs(Xfm)), 1.1*max(abs(Xfm))])

    # Subplot 2: Espectro del mensaje
    plt.subplot(2, 1, 2)
    plt.plot(vfre, abs(Xfm), label='Mensaje')
    plt.xlabel('$f [Hz]$')
    plt.ylabel('$|X(f)|$')
    plt.legend()
    plt.ylim([min(abs(Xfm)), 1.1*max(abs(Xfm))])

    st.pyplot(plt.gcf())  # Mostrar la figura con los subplots
    plt.close()


    # Reproducir el audio
    st.header("Reproducción de Audio")
    st.subheader("Audio de la portadora ")
    st.audio(c, format='audio/wav', sample_rate=fs, start_time=start_time)
    st.subheader("Audio Original")
    st.audio(xs, format='audio/wav', sample_rate=fs, start_time=0)
    st.subheader("Audio Modulado")
    st.audio(y, format='audio/wav', sample_rate=fs, start_time=0)

    st.header("Demodulación")

    y = (1 + (xs / Ac)) * c
    mixer_1=y*c
    analytical_signal = hilbert(y)
    envelope = np.abs(analytical_signal)
    def butter_lowpass(cutoff, Fs, order=6):
        nyq = 0.5 * Fs  # Frecuencia de Nyquist
        normal_cutoff = cutoff / nyq
        if normal_cutoff >= 1:
            normal_cutoff = 0.99
        elif normal_cutoff <= 0:
            normal_cutoff = 0.01

        sos = butter(order, normal_cutoff, btype='low', analog=False, output='sos')
        return sos

    def butter_lowpass_filt(data, cutoff, Fs, order=6):
        sos = butter_lowpass(cutoff, Fs, order)
        y = sosfilt(sos, data)
        return y

    cutoff = 10000
    fin = butter_lowpass_filt(envelope, cutoff, Fs)

    fin -= np.mean(fin)
    Xfmi = np.fft.rfft(mixer_1)

    amplitude_ratio = np.max(np.abs(xs)) / np.max(np.abs(fin))
    filt = fin * amplitude_ratio
    filt_2 = fin * amplitude_ratio * 1.5
    Xfilt=np.fft.rfft(filt_2)
    Xfin = np.fft.rfft(fin)
    st.subheader("Mixer")
    st.markdown("Paso por mixer en el espectro")
    plt.figure(figsize=(8, 6))
    # Subplot 1: Señal Modulada
    plt.subplot(2, 1, 1)
    plt.plot(vfre, abs(Xfy), label='modulado', color="r")
    plt.legend()
    plt.xlabel('$f [Hz]$')
    plt.ylabel('$|X(f)|$')
    plt.ylim([min(abs(Xfm)), 1.1 * max(abs(Xfm))])

    # Subplot 2: Señal Mixer
    plt.subplot(2, 1, 2)
    plt.plot(vfre, abs(Xfmi), label='mixer')
    plt.legend()
    plt.xlabel('$f [Hz]$')
    plt.ylabel('$|X(f)|$')
    plt.ylim([min(abs(Xfm)), 1.1 * max(abs(Xfm))])

    # Mostrar la figura con los subplots en Streamlit
    st.pyplot(plt.gcf())  # .gcf() obtiene la figura actual
    plt.close()
    st.markdown("Paso por el mixer en el tiempo")
    plt.figure(figsize=(10, 8))
    # Subplot 1: Señal Modulada AM
    plt.subplot(3, 1, 1)
    plt.plot(t, y)
    plt.title("Señal Modulada AM")
    plt.xlabel('Tiempo [s]')
    plt.ylabel('Amplitud')

    # Subplot 2: Señal Mixer
    plt.subplot(3, 1, 2)
    plt.plot(t, mixer_1)
    plt.title("Mixer")
    plt.xlabel('Tiempo [s]')
    plt.ylabel('Amplitud')

    # Ajustar diseño para evitar solapamientos
    plt.tight_layout()

    # Mostrar la figura en Streamlit
    st.pyplot(plt.gcf())
    plt.close()
    st.subheader("Audio paso por mixer")
    st.audio(mixer_1, format='audio/wav', sample_rate=fs, start_time=0)
    st.subheader("Filtrado")
    st.markdown("Paso por el filtro pasa bajas en el espectro")
    plt.figure(figsize=(8, 6))
    # Subplot 1: Señal Modulada
    plt.subplot(2, 1, 1)
    plt.plot(vfre, abs(Xfmi), label='Mixer', color="r")
    plt.legend()
    plt.xlabel('$f [Hz]$')
    plt.ylabel('$|X(f)|$')
    plt.ylim([min(abs(Xfm)), 1.1 * max(abs(Xfm))])

    # Subplot 2: Señal Mixer
    plt.subplot(2, 1, 2)
    plt.plot(vfre, abs(Xfilt), label='Filtrado')
    plt.legend()
    plt.xlabel('$f [Hz]$')
    plt.ylabel('$|X(f)|$')
    plt.ylim([min(abs(Xfm)), 1.1 * max(abs(Xfm))])

    # Mostrar la figura con los subplots en Streamlit
    st.pyplot(plt.gcf())  # .gcf() obtiene la figura actual
    plt.close()

    st.markdown("Paso por el filtro pasa bajas en el tiempo")
    plt.figure(figsize=(10, 8))
    # Subplot 1: Señal Modulada AM
    plt.subplot(3, 1, 1)
    plt.plot(t, mixer_1)
    plt.title("Mixer")
    plt.xlabel('Tiempo [s]')
    plt.ylabel('Amplitud')

    # Subplot 2: Señal Mixer
    plt.subplot(3, 1, 2)
    plt.plot(t, filt_2)
    plt.title("Filtrado")
    plt.xlabel('Tiempo [s]')
    plt.ylabel('Amplitud')
    st.subheader("Audio paso por filtro")
    st.audio(filt_2, format='audio/wav', sample_rate=fs, start_time=0)
    st.subheader("Filtrado y escalado")
    st.markdown("Filtrado y escalado en el espectro")
    plt.figure(figsize=(8, 6))
    # Subplot 1: Señal Modulada
    plt.subplot(2, 1, 1)
    plt.plot(vfre, abs(Xfilt), label='filtrado', color="r")
    plt.legend()
    plt.xlabel('$f [Hz]$')
    plt.ylabel('$|X(f)|$')
    plt.ylim([min(abs(Xfm)), 1.1 * max(abs(Xfm))])

    # Subplot 2: Señal Mixer
    plt.subplot(2, 1, 2)
    plt.plot(vfre, abs(Xfin), label='Filtrado y escalado')
    plt.legend()
    plt.xlabel('$f [Hz]$')
    plt.ylabel('$|X(f)|$')
    plt.ylim([min(abs(Xfm)), 1.1 * max(abs(Xfm))])

    # Mostrar la figura con los subplots en Streamlit
    st.pyplot(plt.gcf())  # .gcf() obtiene la figura actual
    plt.close()
    st.markdown("Filtrado y escalado en el tiempo")
    plt.figure(figsize=(10, 8))
    # Subplot 1: Señal Modulada AM
    plt.subplot(3, 1, 1)
    plt.plot(t, filt_2)
    plt.title("Filtrado")
    plt.xlabel('Tiempo [s]')
    plt.ylabel('Amplitud')

    # Subplot 2: Señal Mixer
    plt.subplot(3, 1, 2)
    plt.plot(t, fin)
    plt.title("Filtrado y escalado")
    plt.xlabel('Tiempo [s]')
    plt.ylabel('Amplitud')
    st.subheader("Comparacion entre el mensaje original y el mensaje recuperado")
    st.markdown("Grafica del mensaje original y el mensaje recuperado en el espectro")
    plt.figure(figsize=(8, 6))
    # Subplot 1: Señal Modulada
    plt.subplot(2, 1, 1)
    plt.plot(vfre, abs(Xfin), label='Recuperado', color="r")
    plt.legend()
    plt.xlabel('$f [Hz]$')
    plt.ylabel('$|X(f)|$')
    plt.ylim([min(abs(Xfm)), 1.1 * max(abs(Xfm))])

    # Subplot 2: Señal Mixer
    plt.subplot(2, 1, 2)
    plt.plot(vfre, abs(Xfm), label='Mensaje')
    plt.legend()
    plt.xlabel('$f [Hz]$')
    plt.ylabel('$|X(f)|$')
    plt.ylim([min(abs(Xfm)), 1.1 * max(abs(Xfm))])

    # Mostrar la figura con los subplots en Streamlit
    st.pyplot(plt.gcf())
    plt.close()
    st.markdown("Grafica del mensaje original y el mensaje recuperado en el tiempo")
    plt.figure(figsize=(10, 8))
    # Subplot 1: Señal Modulada AM
    plt.subplot(3, 1, 1)
    plt.plot(t, fin)
    plt.title("Recuperado")
    plt.xlabel('Tiempo [s]')
    plt.ylabel('Amplitud')

    # Subplot 2: Señal Mixer
    plt.subplot(3, 1, 2)
    plt.plot(t, xs)
    plt.title("Mensaje")
    plt.xlabel('Tiempo [s]')
    plt.ylabel('Amplitud')
    st.subheader("Audio recuperado")
    st.audio(fin, format='audio/wav', sample_rate=fs, start_time=0)
    @st.cache_resource
    def cargar_modelo():
        archivo_id = '1LlQzilM3QZUpLJDQzIdbT7rsrTmlOkk0'
        url = f'https://drive.google.com/uc?id={archivo_id}'
        salida = 'modelo.pkl'
        gdown.download(url, salida, quiet=False)
        modelo = joblib.load(salida)
        return modelo

    def procesar_audio(senal, longitud_esperada):
        if len(senal) < longitud_esperada:
            senal = np.pad(senal, (0, longitud_esperada - len(senal)), 'constant')
        else:
            senal = senal[:longitud_esperada]
        return senal

    def calcular_fft_normalizada(senal, escalador, longitud_esperada):
        fft_senal = np.fft.rfft(senal)
        fft_abs = np.abs(fft_senal)

        if len(fft_abs) < longitud_esperada:
            fft_abs = np.pad(fft_abs, (0, longitud_esperada - len(fft_abs)), 'constant')
        else:
            fft_abs = fft_abs[:longitud_esperada]

        fft_abs = fft_abs.reshape(1, -1)
        fft_abs_normalizada = escalador.transform(fft_abs).flatten()

        return fft_abs_normalizada

    def clasificar_genero(Xw_nueva, Xw, etiquetas, fmax, metrica='cosine'):
        distancias = cdist([Xw_nueva[:fmax]], Xw[:, :fmax], metric=metrica)
        etiquetas_predichas = [etiquetas[np.argmin(d)] for d in distancias]

        # Convertir etiquetas a una lista regular de números, no arrays de NumPy
        etiquetas_predichas = list(map(int, etiquetas_predichas))
        genero_predicho = max(set(etiquetas_predichas), key=etiquetas_predichas.count)

        return genero_predicho

    def detectar_genero(genero_predicho):
        if genero_predicho == 1:
            return "Clásica"
        elif genero_predicho == 2:
            return "Metal"
        else:
            return "Género desconocido"

    st.title('Detección de Género Musical')

    modelo = cargar_modelo()
    Xw = modelo['Xw_']
    fmax = modelo['fmax']
    etiquetas = modelo['label']
    fs = modelo['fs']
    escalador = MinMaxScaler().fit(Xw)

    longitud_esperada = len(Xw[0])

    senal_fin_procesada = procesar_audio(fin, longitud_esperada)
    Xw_fin_nueva = calcular_fft_normalizada(senal_fin_procesada, escalador, longitud_esperada)
    genero_fin_predicho = clasificar_genero(Xw_fin_nueva, Xw, etiquetas, fmax)
    resultado_final = detectar_genero(genero_fin_predicho)

    st.write(f"El género es: {resultado_final}")



Writing Demodulacion2.py


In [ ]:
token = '2maC7FWeV10FqgyYS7bbT7LlG31_VL9v3A7SAuVTyZMyKP12' #colocar aquí su token personal después de crear su cuenta con correo UNAL en Ngrok

In [ ]:
from pyngrok import ngrok
# Set authentication token (unique per user)
ngrok.set_auth_token(token)

# Start Streamlit server on a specific port
!nohup streamlit run Demodulacion2.py --server.port 5011 &

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://f931-34-19-29-98.ngrok-free.app
